# Helper tools to process data for their use in the tool

In [ ]:
from music21 import *
import predominantmelodymakam as pmm
import intonation
import numpy as np
import json
import os

## Search patterns in score
It requires a score with just the section corresponding to the recording to be used in the tool

In [ ]:
filefolder = '../../../6fe7108c-4e4f-457b-a363-ccf505bdee9a'
filename = '6fe7108c-4e4f-457b-a363-ccf505bdee9a-ma1.musicxml'

filepath = os.path.join(filefolder, filename)
print(filepath)

In [ ]:
patterns = {
    'alistihlal': [
        (3, 'G4F4E4'),
        (3, 'E4F4G4'),
        (3, 'F4A4G4'),
        (6, 'G4A4B4C4A4G4'),
        (3, 'A3B3C4'),
        (4, 'F4E4D4C4')
    ],
    'almaya': [
        (3, 'B-4A4G4'),
        (3, 'E4F4G4'),
        (3, 'B3D4C4'),
        (2, 'B3C4'),
        (8, 'E4F4G4E4F4E4D4C4'),
        (4, 'G4F4E4')]
}

colors = ['red', 'blue', 'green', 'fuchsia', 'aqua', 'olive', 'maroon', 'navy', 'teal', 'purple', 'lime', 'yellow']

In [ ]:
s = converter.parse(filepath)
p = s.parts[0]
nr = p.flat.notesAndRests.stream()

Search patterns for the selected tab'

In [ ]:
tab = 'alistihlal'

print('Patterns found:')

found_patterns = {}

position = 'above'

for pattern in patterns[tab]:
    pl = pattern[0] # pattern length
    ps = pattern[1] # pattern sequence
    color = colors[patterns[tab].index(pattern)]
    occurrences = 0
    for i in range(len(nr)-pl):
        allNotes = True
        sequence = ''
        for j in range(pl):
            n = nr[i + j]
            if n.isNote:
                sequence += n.nameWithOctave
            else:
                allNotes = False
        if allNotes and ps == sequence:
            occurrences += 1
            found_patterns[i] = (pl, position)
            for j in range(pl):
                n = nr[i + j]
                n.style.color = color
    if position == 'above':
        position = 'below'
    elif position == 'below':
        position = 'above'
            
    print('  {}: {} ({})'.format(ps, occurrences, color))

# s.show()

Add bracket lines for the found patterns

In [ ]:
for k in sorted(found_patterns.keys()):
    line = spanner.Slur()
    for i in range(found_patterns[k][0]):
        n = nr[k + i]
        line.addSpannedElements(n)
    line.placement = found_patterns[k][1]
    p.insert(line)

s.show()

## Beats
Transform SV annotation layer to json object

In [ ]:
filefolder = '../../../6fe7108c-4e4f-457b-a363-ccf505bdee9a'
filename = '6fe7108c-4e4f-457b-a363-ccf505bdee9a-ma1-beats.txt'

filepath = os.path.join(filefolder, filename)
print(filepath)

In [ ]:
with open(filepath, 'r') as f:
    data = f.readlines()

In [ ]:
mizan = 'btayhi' # basit, qaim_wa_nisf, btayhi, darj, quddam

tak = []
dum = []

for d in data:
    time = round(float(d.split('\t')[0]), 2)
    beat = d.rstrip()[-1]
    if mizan == 'basit':
        pass
    elif mizan == 'qaim_wa_nisf':
        pass
    elif mizan == 'btayhi':
        if beat == '1' or beat == '4' or beat == '7':
            dum.append(time)
        else:
            tak.append(time)
    elif mizan == 'darj':
        pass
    elif mizan == 'quddam':
        if beat == '3':
            dum.append(time)
        else:
            tak.append(time)

len(data) == len(tak) + len(dum)

In [ ]:
print(json.dumps({'tak':tak,'dum':dum}, indent=2))

## Patterns
Transform SV annotation layer to json object

In [ ]:
filefolder = '../../../6fe7108c-4e4f-457b-a363-ccf505bdee9a'
filename = '6fe7108c-4e4f-457b-a363-ccf505bdee9a-ma1-patterns.txt'

filepath = os.path.join(filefolder, filename)
print(filepath)

In [ ]:
with open(filepath, 'r') as f:
    data = f.readlines()

In [ ]:
patterns = {}

for d in data:
    start = round(float(d.split('\t')[0]), 2)
    end = round(float(d.split('\t')[0]) + float(d.split('\t')[2]), 2)
    pattern = d.split('\t')[-1].rstrip()
    if pattern[0] != 'C':
        print('Annotation error at', start)
    if pattern not in patterns:
        patterns[pattern] = [{'start':start, 'end':end}]
    else:
        patterns[pattern].append({'start':start, 'end':end})

In [ ]:
print(json.dumps(patterns, indent=2))

## Extract pitch track

In [ ]:
filefolder = '../../../6fe7108c-4e4f-457b-a363-ccf505bdee9a'
filename = '6fe7108c-4e4f-457b-a363-ccf505bdee9a-ma1.mp3'

filepath = os.path.join(filefolder, filename)
print(filepath)

In [ ]:
# Parameters: min_frequency, max_frequency, filter_pitch (bool), peak_distribution_threshold
# extractor = pmm.PredominantMelodyMakam(min_frequency=100, max_frequency=340, filter_pitch=False, peak_distribution_threshold=2)
extractor = pmm.PredominantMelodyMakam(min_frequency=200, max_frequency=550, peak_distribution_threshold=2)
pitchTrack = extractor.run(filepath)

In [ ]:
pitchTrackTxt = ''
for i in pitchTrack['pitch']:
    if i[1] == 0:
        p = '0'
    else:
        p = '{:.6f}'.format(i[1])
    pitchTrackTxt += '{:.6f}\t{}\n'.format(i[0], p)

In [ ]:
minf0 = pitchTrack['settings']['minFrequency']
maxf0 = pitchTrack['settings']['maxFrequency']
pdt = pitchTrack['settings']['peakDistributionThreshold']
fp = pitchTrack['settings']['filterPitch']
filePath2write = filepath[:-4] + '-pmm{}-{}-{}-{}.txt'.format(minf0, maxf0, pdt, fp)
with open(filePath2write, 'w') as f:
    f.write(pitchTrackTxt.rstrip())

## Generate json pitch track

In [ ]:
filefolder = '../../../6fe7108c-4e4f-457b-a363-ccf505bdee9a'
filename = '6fe7108c-4e4f-457b-a363-ccf505bdee9a-ma1-pmm100-250-2-True.txt'

filepath = os.path.join(filefolder, filename)
print(filepath)

In [ ]:
pitchTrack = np.loadtxt(filepath)

In [ ]:
f0 = pitchTrack[:,1]
timeStamps = pitchTrack[:,0]
pitch_obj = intonation.Pitch(timeStamps[f0>0], f0[f0>0])

In [ ]:
rec_obj = intonation.Recording(pitch_obj)

In [ ]:
bins = 1900
histogram_computed = False

while not histogram_computed:
    try:
        rec_obj.compute_hist(bins=bins)
        rec_obj.histogram.get_peaks()
        rec_obj.histogram.plot()
        print('Histogram computed with {} bins.'.format(bins))

        peaks = rec_obj.histogram.peaks['peaks']
        for i in range(len(peaks[0])):
            print('{}. {} : {:.6f}'.format(i, peaks[0][i], peaks[1][i]))
        histogram_computed = True
    except RuntimeError:
        bins += -1

In [ ]:
hist_tonic_index = 5
hist_tonic = rec_obj.histogram.peaks['peaks'][0][hist_tonic_index]
print(hist_tonic)

In [ ]:
tonic = 0
distance = 1

for p in f0[f0>0]:
    current_distance = np.abs(p - hist_tonic)
    if current_distance < distance:
        tonic = p
        distance = current_distance
        
print(tonic)

In [ ]:
f0cents = 1200 * np.log2(f0[f0>0] / hist_tonic)

In [ ]:
pitch_obj = intonation.Pitch(timeStamps[f0>0], f0cents)

rec_obj = intonation.Recording(pitch_obj)
rec_obj.compute_hist(bins=bins)
rec_obj.histogram.get_peaks()
rec_obj.histogram.plot()
peaks = rec_obj.histogram.peaks['peaks']
for i in range(len(peaks[0])):
    print('{}. {} : {:.6f}'.format(i, peaks[0][i], peaks[1][i]))

In [ ]:
pitchTrackDict = {}

for i in range(len(f0)):
    key = '{:.2f}'.format(timeStamps[i])
    if f0[i] == 0:
        value = 's'
    else:
        value = np.round(1200 * np.log2(f0[i] / hist_tonic))
    pitchTrackDict[key] = value

In [ ]:
print(json.dumps(pitchTrackDict))